
# Bring your own Trainer to Clara FL 

In V4 Clara FL allows you to bring your own trainer that could be anything.
Therefore it does not need to use clara train. 
This Notebook shows how to use clara FL using a MONAL trainer.
Following this steps you could then change it to bring your own Tensorflow trainer 
or any Non Deep learning algorithm 


## Resources
You could watch the free GTC 2021 talks covering Clara Train SDK
- [Clara Train 4.0 - 101 Getting Started [SE2688]](https://gtc21.event.nvidia.com/media/Clara%20Train%204.0%20-%20101%20Getting%20Started%20%5BSE2688%5D/1_0qgfrql2)
- [Clara Train 4.0 - 201 Federated Learning [SE3208]](https://gtc21.event.nvidia.com/media/Clara%20Train%204.0%20-%20201%20Federated%20Learning%20%5BSE3208%5D/1_m48t6b3y)
- [What’s New in Clara Train 4.0 [D3114]](https://gtc21.event.nvidia.com/media/What%E2%80%99s%20New%20in%20Clara%20Train%204.0%20%5BD3114%5D/1_umvjidt2)
- [Take Medical AI from Concept to Production using Clara Imaging [S32482]](https://gtc21.event.nvidia.com/media/Take%20Medical%20AI%20from%20Concept%20to%20Production%20using%20Clara%20Imaging%20%20%5BS32482%5D/1_6bvnvyg7)
- [Federated Learning for Medical AI [S32530]](https://gtc21.event.nvidia.com/media/Federated%20Learning%20for%20Medical%20AI%20%5BS32530%5D/1_z26u15uk)
- [Get Started Now on Medical Imaging AI with Clara Train on Google Cloud Platform [S32518]](https://gtc21.event.nvidia.com/media/Get%20Started%20Now%20on%20Medical%20Imaging%20AI%20with%20Clara%20Train%20on%20Google%20Cloud%20Platform%20%5BS32518%5D/1_2yjdekmi)
- [Automate 3D Medical Imaging Segmentation with AutoML and Neural Architecture Search [S32083]](https://gtc21.event.nvidia.com/media/Automate%203D%20Medical%20Imaging%20Segmentation%20with%20AutoML%20and%20Neural%20Architecture%20Search%20%5BS32083%5D/1_r5swh2jn)
- [A Platform for Rapid Development and Clinical Translation of ML Models for Applications in Radiology at UCSF [S31619]](https://gtc21.event.nvidia.com/media/A%20Platform%20for%20Rapid%20Development%20and%20Clinical%20Translation%20of%20ML%20Models%20for%20Applications%20in%20Radiology%20at%20UCSF%20%5BS31619%5D/1_oz8qop5a)


## 1. Brief Introduction

We are providing scripts to get started with BYO Trainer in a similar experience as we showed using clara train sdk.
If you are using clara train sdk docker image, you will need to shut it down.
You would follow steps below that will launch a new jupyter lab based on MONAI docker and not clara train 


## 2. Build Docker Image 

We need to build a docker image for the experiment. 
This docker should contain the framework that would run on the client and server. 
You simply need to run `startDocker.sh` this will:
- build a docker image based on MONAI's latest docker image in dockerhub
- Install the fl wheel file 
- Install zip utility
- tag the image as `projectmonai/monaiwfl:0.1`

Steps above are controlled from the [Dockerfile](FLBYOTrainer/docker/Dockerfile)
 
In the end the `startDocker.sh` will launch a jupyter lab you should see output as 
```
[I 16:49:04.476 LabApp] Serving notebooks from local directory: /claraDevDay
[I 16:49:04.476 LabApp] Jupyter Notebook 6.2.0 is running at:
[I 16:49:04.476 LabApp] http://hostname:8888/?token=43f3dbe2acfab1f4412279d7a7b40d8471e5c966845299ea
[I 16:49:04.476 LabApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 16:49:04.480 LabApp] 
    To access the notebook, open this file in a browser:
        file:///root/.local/share/jupyter/runtime/nbserver-1-open.html
    Or copy and paste this URL:
        http://hostname:8888/?token=43f3dbe2acfab1f4412279d7a7b40d8471e5c966845299ea
```


## 3. Provision Package Preparation

You can now use the `provision` command to provision your FL experiment.
You can still use the same [Provisioning Notebook](../Provisioning.ipynb) to generate the packages
Please note that you are in a different container than clara train so you would need to redo the provisioning step.


## 4. Run Admin client 

One you extracted the packages, you can follow along the [Admin Notebook](Admin.ipynb).
Only difference is that you upload the [adminMMAR_BYOT_monai Folder](adminMMAR_BYOT_monai) instead of [adminMMAR Folder](adminMMAR)

In case you would like to write your own trainer 
to work with Tensor Flow or non deep learning algorithms,
You should checkout the Trainer class

```
class MONAITrainer(Trainer):
    """
    This class implements a MONAI based trainer that can be used for Federated Learning.
    Args:
        aggregation_epochs: the number of training epochs for a round.
            This parameter only works when `aggregation_iters` is 0. Defaults to 1.
        aggregation_iters:  the number of training iterations for a round.
            If the value is larger than 0, the trainer will use iteration based aggregation
            rather than epoch based aggregation. Defaults to 0.
    """

    def __init__(self, aggregation_epochs: int = 1, aggregation_iters: int = 0):

    def _initialize_trainer(self, fl_ctx: FLContext):
        """
        The trainer's initialization function. At the beginning of a FL experiment,
        the train and evaluate engines, as well as train context and FL context
        should be initialized.
        """

    def handle_event(self, event_type: str, fl_ctx: FLContext):
        """
        This function is an extended function from the super class.
        It is used to perform the handler process based on the
        event_type. At the start point of a FL experiment, necessary
        components should be initialized. At the end of the experiment,
        the running engines should be terminated.

        Args:
            event_type: the type of event that will be fired. In MONAITrainer,
                only `START_RUN` and `END_RUN` need to be handled.
            fl_ctx: an `FLContext` object.
        """
    def train(self, shareable: Shareable, fl_ctx: FLContext, abort_signal: Signal) -> Shareable:
        """
        This function is an extended function from the super class.
        As a supervised learning based trainer, the train function will run
        evaluate and train engines based on model weights from `shareable`.
        After fininshing training, a new `Shareable` object will be submitted
        to server for aggregation.

        Args:
            shareable: the `Shareable` object acheived from server.
            fl_ctx: the `FLContext` object achieved from server.
            abort_signal: if triggered, the training will be aborted.

        Returns:
            a new `Shareable` object to be submitted to server for aggregation.
        """
```
